# Generate a social compliance index for every individual

For every individual, sum up visits to 4 categories in 2019.3.15-2019.4.15 & 2020.3.15-2020.4.15 and get the ratio

In [0]:
visit_2019_2020.display()

In [0]:
test = visit_2019_2020.filter(f.col("venue_category").isin((food_cat + night_spot_cat + art_entertainment_cat + event_cat)))
grouped2019 = test.filter((f.col("arrive_year") == 2019) & (f.col("utc_arrived_date").between("2019-03-15", "2019-04-15"))).groupBy(["panelist_id", "arrive_year"]).count().withColumnRenamed("count", "cnt_2019")
grouped2020 = test.filter((f.col("arrive_year") == 2020) & (f.col("utc_arrived_date").between("2020-03-15", "2020-04-15"))).groupBy(["panelist_id", "arrive_year"]).count().withColumnRenamed("count", "cnt_2020")
df = grouped2019.join(grouped2020, on=["panelist_id"])
df = df.withColumn("social_compliance_index", f.round(f.col("cnt_2020")/f.col("cnt_2019"), 4))
df.display()

In [0]:
df.display()

panelist_id,utc_arrived_date,arrive_year,arrive_month_date,cnt_2019,cnt_2020


#### Check for outdoor categories
- level 0: Outdoors & Recreation
- level 1: Big Box Store, Drugstore, Grocery Store, Market, Organic Grocery, Pharmacy, Supermarket

In [0]:
outdoor_cat = parent_categorization.filter(f.col("level_0_name") == "Outdoors & Recreation").select("category").rdd.flatMap(lambda x: x).collect()

In [0]:
visit_2019_2020.filter(f.col("venue_category").isin(outdoor_cat)).display()

panelist_venue_visit_sk,panelist_id,utc_arrived_date,utc_arrived_at,local_arrived_at,utc_departed_at,local_departed_at,local_arrivaled_date_sk,local_departed_date_sk,Time_of_the_day,Duration,confidence,venue_id,venue_name,venue_category,venue_category_2,venue_category_3,venue_category_4,venue_category_5,venue_chain_name,venue_chain_name_2,lat,lng,timezone,address,city,state,postal_code,country,dma_name,msa_name,visit_permutation_hash,invocation_id,loaded_at,loaded_by,updated_at,updated_by,arrive_year,arrive_month_date


In [0]:
# level 0: Outdoors & Recreation
df1 = visit_2019_2020.filter((f.col("venue_category").isin(outdoor_cat)) & (f.col("utc_arrived_date").between("2019-03-15", "2019-04-15"))).groupBy('utc_arrived_date', 'arrive_month_date').count().orderBy('arrive_month_date', ascending=True).toPandas()
df2 = visit_2019_2020.filter((f.col("venue_category").isin(outdoor_cat)) & (f.col("utc_arrived_date").between("2020-03-15", "2020-04-15"))).groupBy('utc_arrived_date', 'arrive_month_date').count().orderBy('arrive_month_date', ascending=True).toPandas()

plt.figure(figsize=(12,8))
sns.lineplot(x = "arrive_month_date", y = "count", linewidth=1.5, data = df1, color='b', label="2019.3.15-2019.4.15")
sns.lineplot(x = "arrive_month_date", y = "count", linewidth=1.5, data = df2, color='r', label="2020.3.15-2020.4.15")
plt.xticks(rotation = 45)
plt.title("Level 0: Outdoors & Recreation", fontsize=16)
plt.legend()
plt.show()

In [0]:
# Level 1: Big Box Store, Drugstore, Grocery Store, Market, Organic Grocery, Pharmacy, Supermarket
# make a loop to plot
plt.figure(figsize=(20,30))
level1_list = ["Big Box Store", "Drugstore", "Grocery Store", "Market", "Organic Grocery", "Pharmacy", "Supermarket"]
i=1
for cat in level1_list:
    df1 = visit_2019_2020.filter(((f.col("venue_category_2") == cat) | (f.col("venue_category") == cat)) & (f.col("utc_arrived_date").between("2019-03-15", "2019-04-15"))).groupBy('utc_arrived_date', 'arrive_month_date').count().orderBy('arrive_month_date', ascending=True).toPandas()
    df2 = visit_2019_2020.filter(((f.col("venue_category_2") == cat) | (f.col("venue_category") == cat)) & (f.col("utc_arrived_date").between("2020-03-15", "2020-04-15"))).groupBy('utc_arrived_date', 'arrive_month_date').count().orderBy('arrive_month_date', ascending=True).toPandas()
    plt.subplot(4,2,i).set_title(f"Category: {cat}")
    sns.lineplot(x = "arrive_month_date", y = "count", linewidth=1.5, data = df1, color='b', label="2019.3.15-2019.4.15")
    sns.lineplot(x = "arrive_month_date", y = "count", linewidth=1.5, data = df2, color='r', label="2020.3.15-2020.4.15")
    plt.xticks(rotation="vertical")
    plt.legend()
    i = i+1    
plt.show()

# Modified Oct 15: Plotting per capita visit for main categories

In [0]:
# generate the number of unique users over the past 30 days

from pyspark.sql.window import Window
w = (Window()
   .partitionBy(f.col("utc_arrived_date"))
   .orderBy(f.col("utc_arrived_date").cast("timestamp").cast("long"))
   .rowsBetween(-30*86400, 0))

test = visit_2019_2020.withColumn('distinct_panelist_over_past_30days', f.approx_count_distinct("panelist_id").over(w))
test.display()


In [0]:
# testify

visit_2019_2020.filter(f.col("utc_arrived_date").between("2019-09-16", "2019-09-22")).select(f.countDistinct("panelist_id")).show()

+---------------------------+
count(DISTINCT panelist_id)|
+---------------------------+
 129071|
+---------------------------+

In [0]:
test.orderBy("utc_arrived_date", ascending=True).display()

In [0]:
visit_2019_2020.display()

### generate monthly unique panelist id (by parent category)

In [0]:
# denominator: unique panelist in the current month for each category

month_distinct = visit_2018_to_2020.groupBy("arrive_year_month", "parent_category").agg(f.countDistinct("panelist_id")).withColumnRenamed("count(panelist_id)", "monthly_unique_panelist").orderBy("arrive_year_month", ascending=True)
visit_2018_to_2020 = visit_2018_to_2020.join(month_distinct, on=["arrive_year_month", "parent_category"]).orderBy('utc_arrived_date', ascending=True)
#visit_2018_to_2020.display()

## Category: Food

In [0]:
visit_2018_to_2020.groupBy("arrive_year_month").agg(f.countDistinct("panelist_id")).withColumnRenamed("count(panelist_id)", "monthly_unique_panelist").orderBy("arrive_year_month", ascending=True).display()

arrive_year_month,monthly_unique_panelist
2018-01,148014
2018-02,147197
2018-03,133726
2018-04,145035
2018-05,146132
2018-06,140113
2018-07,147368
2018-08,164577
2018-09,168605
2018-10,176488


In [0]:
# generate "per capita" visit for each category in each day

food = visit_2018_to_2020.filter(f.col("venue_category").isin(food_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)

food = food.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

food.display()

In [0]:
# plotting for food
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

df_2018 = food.filter(f.col('arrive_year') == 2018).orderBy('arrive_month_date', ascending=True).toPandas()
df_2019 = food.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
df_2020 = food.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2018, color='g', label="Year 2018", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.set_title("Visit per capita: Food", fontsize=24)
plt.show()

## Category: Nightlife spot

In [0]:
# generate "per capita" visit for each category in each day

night_spot = visit_2018_to_2020.filter(f.col("venue_category").isin(night_spot_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)

night_spot = night_spot.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

night_spot.display()

In [0]:
# plotting for nightlife spot

df_2018 = night_spot.filter(f.col('arrive_year') == 2018).orderBy('arrive_month_date', ascending=True).toPandas()
df_2019 = night_spot.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
df_2020 = night_spot.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      
fig, ax = plt.subplots(figsize=(12,8))
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2018, color='g', label="Year 2018", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.set_title("Visit per capita: Nightlife Spot", fontsize=24)
plt.show()

## Category: Art & Entertainment

In [0]:
# generate "per capita" visit for each category in each day

art_entertainment = visit_2018_to_2020.filter(f.col("venue_category").isin(art_entertainment_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)

art_entertainment = art_entertainment.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

art_entertainment.display()

In [0]:
# plotting for art&entertainment

df_2018 = art_entertainment.filter(f.col('arrive_year') == 2018).orderBy('arrive_month_date', ascending=True).toPandas()
df_2019 = art_entertainment.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
df_2020 = art_entertainment.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2018, color='g', label="Year 2018", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.set_title("Visit per capita: Art & Entertainment", fontsize=24)
plt.show()

## Category: Outdoor & Recreation

In [0]:
# generate "per capita" visit for each category in each day
outdoor = visit_2018_to_2020.filter(f.col("venue_category").isin(outdoor_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
outdoor = outdoor.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))
outdoor.display()

In [0]:
# plotting for outdoor & recreation
import matplotlib.ticker as ticker

df_2018 = outdoor.filter(f.col('arrive_year') == 2018).orderBy('arrive_month_date', ascending=True).toPandas()
df_2019 = outdoor.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
df_2020 = outdoor.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2018, color='g', label="Year 2018", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.set_title("Visit per capita: Outdoor & Recreation", fontsize=24)
plt.show()

## Category: All other except for transportation

In [0]:
# generate "per capita" visit for each category in each day
others = visit_2018_to_2020.filter(f.col("venue_category").isin(others)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
others = others.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))
others.display()

In [0]:
# plotting for other categories

df_2018 = others.filter(f.col('arrive_year') == 2018).orderBy('arrive_month_date', ascending=True).toPandas()
df_2019 = others.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
df_2020 = others.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      
fig, ax = plt.subplots(figsize=(12,8))
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2018, color='g', label="Year 2018", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.set_title("Visit per capita: Others (excluding transportation)", fontsize=24)
plt.show()

# Update social compliance index

In [0]:
%run "/Users/shining.yang@duke.edu/COVID_policy"

In [0]:
visit_2019_2020 = visit_2018_to_2020.filter(f.col("arrive_year").isin(["2019", "2020"]))
visit_covid = visit_2019_2020.join(test, visit.state == test.State_Tribe_Territory, "left").drop("State_Tribe_Territory").filter(f.col("state").isNotNull())
#visit_covid.display()

In [0]:
# state missing ratio
np.round(visit_covid.filter(f.col("state").isNull()).count() / visit_2019_2020.count(), 4)

Out[95]: 0.0181

In [0]:
# get date range: 30 days from the earliest policy date
visit_covid = visit_covid.withColumn('policy_date_30', f.date_add(f.col("earliest_policy_date"), 30))

# get date range: 90 days from the earliest policy date
visit_covid = visit_covid.withColumn('policy_date_90', f.date_add(f.col("earliest_policy_date"), 90))

#visit_covid.display()

In [0]:
## testing purpose
# test2 = test.filter((f.col("arrive_year") == 2019) & (f.col("arrive_month_date").between(f.substring('earliest_policy_date', 6, 5), f.substring('policy_date_30', 6, 5))))

# test2.filter(f.col("state") == "AL").display()

In [0]:
# generate social compliance index
test = visit_covid.filter(f.col("venue_category").isin((food_cat + night_spot_cat + art_entertainment_cat + event_cat)))
index_2019_30days = test.filter((f.col("arrive_year") == 2019) & (f.col("arrive_month_date").between(f.substring('earliest_policy_date', 6, 5), f.substring('policy_date_30', 6, 5)))).groupBy(["panelist_id"]).count().withColumnRenamed("count", "cnt_2019_30days")

index_2019_90days = test.filter((f.col("arrive_year") == 2019) & (f.col("arrive_month_date").between(f.substring('earliest_policy_date', 6, 5), f.substring('policy_date_90', 6, 5)))).groupBy(["panelist_id"]).count().withColumnRenamed("count", "cnt_2019_90days")

df1 = index_2019_30days.join(index_2019_90days, on=["panelist_id"])
#df1.display()

In [0]:
index_2020_30days = test.filter((f.col("arrive_year") == 2020) & (f.col("arrive_month_date").between(f.substring('earliest_policy_date', 6, 5), f.substring('policy_date_30', 6, 5)))).groupBy(["panelist_id"]).count().withColumnRenamed("count", "cnt_2020_30days")

index_2020_90days = test.filter((f.col("arrive_year") == 2020) & (f.col("arrive_month_date").between(f.substring('earliest_policy_date', 6, 5), f.substring('policy_date_90', 6, 5)))).groupBy(["panelist_id"]).count().withColumnRenamed("count", "cnt_2020_90days")

df2 = index_2020_30days.join(index_2020_90days, on=["panelist_id"])

In [0]:
df = df1.join(df2, on=["panelist_id"])
df = df.withColumn("social_compliance_index_30days", f.round(f.col("cnt_2020_30days")/f.col("cnt_2019_30days"), 4))
df = df.withColumn("social_compliance_index_90days", f.round(f.col("cnt_2020_90days")/f.col("cnt_2019_90days"), 4))
#df.display()

In [0]:
#df.describe().display()

summary,panelist_id,cnt_2019_30days,cnt_2019_90days,cnt_2020_30days,cnt_2020_90days,social_compliance_index_30days,social_compliance_index_90days
count,30052,30052,30052,30052,30052,30052,30052
mean,3082140.5696792225,23.987488353520565,76.23549181418873,14.921036869426327,48.730966325036604,1.1447065752695331,1.102764178756822
stddev,955306.1458850742,24.231778832246164,66.54126081250003,26.0828025580316,64.27441239410359,4.447472512208821,4.113937960684168
min,2,1,1,1,1,0.0019,0.0015
max,4188895,569,2325,1550,3913,469.0,219.0


In [0]:
quantile_list_30days = df.approxQuantile("social_compliance_index_30days", [0.00, 0.25, 0.5, 0.75, 1.00], 0)
quantile_list_90days = df.approxQuantile("social_compliance_index_90days", [0.00, 0.25, 0.5, 0.75, 1.00], 0)

In [0]:
#quantile_list_30days

Out[61]: [0.0019, 0.2222, 0.5, 1.0, 469.0]

In [0]:
#quantile_list_90days

Out[64]: [0.0015, 0.2807, 0.5455, 0.9683, 219.0]

### add quantiles to social compliance index

In [0]:
df_w_quantile = df.withColumn("index_quantile_30days", f.when(f.col("social_compliance_index_30days").between(quantile_list_30days[0], quantile_list_30days[1]), '1st quantile (25%)').when(f.col("social_compliance_index_30days").between(quantile_list_30days[1], quantile_list_30days[2]), '2nd quantile (50%)').when(f.col("social_compliance_index_30days").between(quantile_list_30days[2], quantile_list_30days[3]), '3rd quantile (75%)').otherwise('4th quantile (100%)'))

df_w_quantile = df_w_quantile.withColumn("index_quantile_90days", f.when(f.col("social_compliance_index_90days").between(quantile_list_90days[0], quantile_list_90days[1]), '1st quantile (25%)').when(f.col("social_compliance_index_90days").between(quantile_list_90days[1], quantile_list_90days[2]), '2nd quantile (50%)').when(f.col("social_compliance_index_90days").between(quantile_list_90days[2], quantile_list_90days[3]), '3rd quantile (75%)').otherwise('4th quantile (100%)'))

#df_w_quantile.display()

In [0]:
df_w_quantile.display()

### write df_w_quantile to file

In [0]:
df_w_quantile.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable('duke_university_sandbox.panelist_with_quantile')

In [0]:
# join quantiles onto the visit_covid df
visit_covid = visit_covid.join(df_w_quantile, visit_covid.panelist_id == df_w_quantile.panelist_id, "left").drop( df_w_quantile.panelist_id)

In [0]:
visit_covid = visit_covid.drop("cnt_2019_30days", "cnt_2019_90days", "cnt_2020_30days", "cnt_2020_90days")
#visit_covid.display()

In [0]:
visit_covid.columns

Out[73]: ['arrive_year_month',
 'parent_category',
 'panelist_venue_visit_sk',
 'pilgrim_visit_id',
 'utc_arrived_date',
 'utc_arrived_at',
 'local_arrived_at',
 'utc_departed_at',
 'local_departed_at',
 'local_arrivaled_date_sk',
 'local_departed_date_sk',
 'Time_of_the_day',
 'Duration',
 'confidence',
 'venue_id',
 'venue_name',
 'venue_category',
 'venue_category_2',
 'venue_category_3',
 'venue_category_4',
 'venue_category_5',
 'venue_chain_name',
 'venue_chain_name_2',
 'lat',
 'lng',
 'timezone',
 'address',
 'city',
 'state',
 'postal_code',
 'country',
 'dma_name',
 'msa_name',
 'invocation_id',
 'loaded_at',
 'loaded_by',
 'updated_date',
 'updated_at',
 'updated_by',
 'arrive_year',
 'arrive_month_date',
 'monthly_unique_panelist',
 'Order_code',
 'Stay_at_Home_Order_Recommendation',
 'earliest_policy_date',
 'policy_date_30',
 'policy_date_90',
 'panelist_id',
 'social_compliance_index_30days',
 'social_compliance_index_90days',
 'index_quantile_30days',
 'index_quantile_90days']

### write to file

In [0]:
visit_covid.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable('duke_university_sandbox.venue_visit_covid')

# Update Oct 15: Plotting user segment

## By covid lockdown order

## Category: Food

In [0]:
# plotting for food
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


orderlist = [["1"], ["2", "3", "4", "5", "6"], ["7"]]
    
plt.subplots(figsize=(20,18))
i = 1
for order in orderlist:
    food = visit_covid.filter((f.col("venue_category").isin(food_cat)) & (f.col("Order_code").isin(order))).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date',  'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
    food = food.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

    df_2019 = food.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
    df_2020 = food.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      

    ax = plt.subplot(2,2,i)
    ax.set_title(f"Category: Food; Order Code = {order}")
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
    i = i+1
    
plt.show()

## Category: Nightlife spot

In [0]:
plt.subplots(figsize=(20,18))
i = 1
for order in orderlist:
    nightlife = visit_covid.filter((f.col("venue_category").isin(night_spot_cat)) & (f.col("Order_code").isin(order))).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date',  'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
    nightlife = nightlife.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

    df_2019 = nightlife.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
    df_2020 = nightlife.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      

    ax = plt.subplot(2,2,i)
    ax.set_title(f"Category: Nightlife Spot; Order Code = {order}")
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
    i = i+1
    
plt.show()

## Category: Art & Entertainment

In [0]:
plt.subplots(figsize=(20,18))
i = 1
for order in orderlist:
    art_entertainment = visit_covid.filter((f.col("venue_category").isin(art_entertainment_cat)) & (f.col("Order_code").isin(order))).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date',  'monthly_unique_panelist').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
    art_entertainment = art_entertainment.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

    df_2019 = art_entertainment.filter(f.col('arrive_year') == 2019).orderBy('arrive_month_date', ascending=True).toPandas()
    df_2020 = art_entertainment.filter(f.col('arrive_year') == 2020).orderBy('arrive_month_date', ascending=True).toPandas()      

    ax = plt.subplot(2,2,i)
    ax.set_title(f"Category: Art & Entertainment; Order Code = {order}")
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2019, color='b', label="Year 2019", ax=ax)
    sns.lineplot(x = "arrive_month_date", y = "visit_per_capita", linewidth=1.5, data = df_2020, color='r', label="Year 2020", ax=ax)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
    i = i+1
    
plt.show()

# Update Nov 6: Merging social compliance index and quantile back to venue visit df

### read in visit_covid

In [0]:
visit_covid = spark.read.load('/user/hive/warehouse/duke_university_sandbox.db/venue_visit_covid/')

In [0]:
visit_covid_filtered = visit_covid.select("utc_arrived_date", "panelist_id", "arrive_year", "arrive_year_month", "arrive_month_date", "Duration", "venue_category", "parent_category", "state", "monthly_unique_panelist", "Order_code", "Stay_at_Home_Order_Recommendation", "earliest_policy_date", "policy_date_30", "policy_date_90", "social_compliance_index_30days", "social_compliance_index_90days", "index_quantile_30days", "index_quantile_90days" ).filter(f.col("index_quantile_30days").isNotNull())

### write the filtered visit_covid df to file

In [0]:
visit_covid_filtered.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable('duke_university_sandbox.venue_visit_covid_filtered')

### read in filtered visit_covid df

In [0]:
visit_covid_filtered = spark.read.load('/user/hive/warehouse/duke_university_sandbox.db/venue_visit_covid_filtered/')

In [0]:
visit_covid_filtered.display()

In [0]:
visit_covid_filtered.count()

Out[6]: 49524270

In [0]:
plt.figure(figsize=(10,8))
df_food = food.toPandas()
sns.lineplot(x = "utc_arrived_date", y = "visit_per_capita", hue = "index_quantile_30days", style="index_quantile_30days", linewidth=1, data = df_food)
plt.title("Category: Food")
plt.show()

In [0]:
visit_covid_df = visit_covid_filtered.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:161: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
 An error occurred while calling o13257.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:428)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:107)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:103)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 74 tasks (7.5 GiB) is bigger than local result size limit 7.5 GiB, to address it, set spark.driver.maxResultSize bigger than your dataset result size.
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3029)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2976)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2970)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2970)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1390)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1390)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1390)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3238)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3179)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3167)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:1152)
	at org.apache.spark.SparkContext.runJobInternal(SparkContext.scala:2651)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2634)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2746)
	at org.apache.spark.sql.Dataset.$anonfun$collectAsArrowToPython$6(Dataset.scala:3887)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1681)
	at org.apache.spark.sql.Dataset.$anonfun$collectAsArrowToPython$3(Dataset.scala:3891)
	at org.apache.spark.sql.Dataset.$anonfun$collectAsArrowToPython$3$adapted(Dataset.scala:3857)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3951)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$8(SQLExecution.scala:239)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:386)
	at org.apache.spark.sql.execution.SQLExecuti

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2569826368122477> in <module> 
 ----> 1 visit_covid_df = visit_covid_filtered . toPandas ( ) 

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in toPandas (self) 
 115 tmp_column_names = [ 'col_{}' . format ( i ) for i in range ( len ( self . columns ) ) ] 
 116 self_destruct = self . sql_ctx . _conf . arrowPySparkSelfDestructEnabled ( ) 
 --> 117 batches = self.toDF(*tmp_column_names)._collect_as_arrow(
 118 split_batches=self_destruct)
 119 if len ( batches ) > 0 : 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in _collect_as_arrow (self, split_batches) 
 294 finally : 
 295 # Join serving thread and raise any exceptions from collectAsArrowToPython 
 --> 296 jsocket_auth_server . getResult ( ) 
 297 
 298 # Separate RecordBatches from batch order indices in results 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o13257.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:428)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:107)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:103)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 74 tasks (7.5 GiB) is bigger than local result size limit 7.5 GiB, to address it, set spark.driver.maxResultSize bigger than your dataset result size.
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3029)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2976)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2970)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBu

In [0]:
import matplotlib.ticker as ticker

# get all parent categories to a list
category_list = visit_covid_filtered.select('parent_category').distinct().rdd.flatMap(list).collect()
category_list

Out[19]: ['Shop & Service',
 'Food',
 'Event',
 'College & University',
 'Professional & Other Places',
 'Travel & Transport',
 'Arts & Entertainment',
 'Nightlife Spot',
 'Outdoors & Recreation',
 'Residence']

In [0]:
import matplotlib.ticker as ticker

# get all parent categories to a list
category_list = visit_covid_filtered.select('parent_category').distinct().rdd.flatMap(list).collect()

for cat in category_list:
    plt.figure(figsize = (10,8))
    #plt.title(f"Category: {cat}", fontsize=12)
    df = visit_covid_filtered.filter(f.col("parent_category") == cat).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist', 'index_quantile_30days').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
    df = df.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))
    df_pd = df.toPandas()
    sns.lineplot(x = "utc_arrived_date", y = "visit_per_capita", hue = "index_quantile_30days", style="index_quantile_30days", linewidth=1, data = df_pd)
    plt.title(f"Category: {cat}")
    
plt.tight_layout()    
plt.show()

In [0]:

# last 5 categories 
category_list = [college_cat,
                 professional_cat,
                 residence_cat,
                 shop_service_cat,
                 transportation_cat]

i = 1
for cat in category_list:
    fig = plt.figure(figsize = (10,8))
    df = visit_covid.filter(f.col("venue_category").isin(cat)).groupBy('arrive_year', 'utc_arrived_date', 'monthly_unique_panelist', 'index_quantile_30days').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
    
    df = df.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))
    df_pd = df.toPandas()
    sns.lineplot(x = "utc_arrived_date", y = "visit_per_capita", hue = "index_quantile_30days", style="index_quantile_30days", linewidth=1, data = df_pd)
    #plt.legend(loc="best", fontsize=12)
    #plt.title(f"Category: {cat}")
    
#plt.tight_layout()    
plt.show()


In [0]:
food = visit_covid.filter(f.col("venue_category").isin(food_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist', 'index_quantile_30days').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
food = food.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))

night = visit_covid.filter(f.col("venue_category").isin(night_spot_cat)).groupBy('arrive_year', 'utc_arrived_date','arrive_year_month', 'arrive_month_date', 'monthly_unique_panelist', 'index_quantile_30days').count().withColumnRenamed("count", "cnt_for_cat").orderBy('utc_arrived_date', ascending=True)
night = night.withColumn("visit_per_capita", f.round(f.col("cnt_for_cat")/f.col("monthly_unique_panelist"), 4))


In [0]:
plt.figure(figsize=(16,12))

subplot(1,2,1)
df_food = food.toPandas()
sns.lineplot(x = "utc_arrived_date", y = "visit_per_capita", hue = "index_quantile_30days", style="index_quantile_30days", linewidth=1, data = df_food)
plt.title("Category: Food")

subplot(1,2,2)
df_night = night.toPandas()
sns.lineplot(x = "utc_arrived_date", y = "visit_per_capita", hue = "index_quantile_30days", style="index_quantile_30days", linewidth=1, data = df_night)
plt.title("Category: Nightlife Spot")
plt.show()